### build-sql-database
Builds database in SQL por from the CPDOC collection **Antonio Azeredo da Silveira, Ministério das Relações Exteriores (AAS-MRE_**.  

The meta-data included are:
* doc_id
* text body
* main language
* readability
* URL

In [14]:
import os
import re
import csv
import sqlite3

from langdetect import detect
from IPython.display import clear_output

# set operational system variables
Verifies what operational system is being used and creates user-specific variables. Renato = Linux ; Marcelo = nt (Windows)

Also sets working folders

In [12]:
inputs = os.path.join("..", "inputs")
outputs = os.path.join("..","outputs")
if os.name == 'nt':
    encoding_type = 'utf-8'
else:
    encoding_type = 'ISO-8859-1'

In [ ]:
path = r"D:\pseudo-dropbox\backups-fgv\textfiles/textfiles-corrected-regrouped/"

### create special sorting function
* Creates function that list files on different order
* It was important to pay atention to files with different numeration, such as :
* AAS_mre_onu_1975.01.23_doc_I-A.txt
* AAS_mre_onu_1975.01.23_doc_I-6A8.txt

In [2]:
def to_zero(x):
    if x == '': x = '0'
    return x
def special_sort(l):
    convert = lambda text: int(text) if text.isdigit() else str(text)
    alphanum_key = lambda key: [ convert(to_zero(c)) for c in filter(None, re.split('(\d)A|A\d|([A-Z]*)-A?|.txt', key))] 
    return sorted(l, key = alphanum_key)

# create list of files with special sort

In [4]:
files = [f for f in sorted(os.listdir(path))]
fullpath_list = []
fullpath = ''
for file in files:
    fullpath = path+file
    fullpath_list.append(fullpath)
fullpath_list = special_sort(fullpath_list)

In [5]:
fullpath_list[0:10]

['D:\\pseudo-dropbox\\backups-fgv\\textfiles/textfiles-corrected-regrouped/AAS_mre_ag_1973.11.20_doc_I-1.txt',
 'D:\\pseudo-dropbox\\backups-fgv\\textfiles/textfiles-corrected-regrouped/AAS_mre_ag_1973.11.20_doc_I-2.txt',
 'D:\\pseudo-dropbox\\backups-fgv\\textfiles/textfiles-corrected-regrouped/AAS_mre_ag_1973.11.20_doc_I-3.txt',
 'D:\\pseudo-dropbox\\backups-fgv\\textfiles/textfiles-corrected-regrouped/AAS_mre_ag_1973.11.20_doc_I-4.txt',
 'D:\\pseudo-dropbox\\backups-fgv\\textfiles/textfiles-corrected-regrouped/AAS_mre_ag_1973.11.20_doc_I-5.txt',
 'D:\\pseudo-dropbox\\backups-fgv\\textfiles/textfiles-corrected-regrouped/AAS_mre_ag_1973.11.20_doc_I-6.txt',
 'D:\\pseudo-dropbox\\backups-fgv\\textfiles/textfiles-corrected-regrouped/AAS_mre_ag_1973.11.20_doc_I-7.txt',
 'D:\\pseudo-dropbox\\backups-fgv\\textfiles/textfiles-corrected-regrouped/AAS_mre_ag_1973.11.20_doc_I-8.txt',
 'D:\\pseudo-dropbox\\backups-fgv\\textfiles/textfiles-corrected-regrouped/AAS_mre_ag_1973.11.20_doc_I-9.txt',
 

# url data

### creates list of dossiers and urls

In [25]:
url_list = []
dossie_url_list = []
url_inputs = os.path.join(inputs,"URLS-AAS-marcelo.csv")
with open(url_inputs, 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=';')
    for row in reader:
        dossie = row[1]
        if not dossie.startswith('AAS mre'): continue
        dossie = re.sub(' ','_',dossie)
        dossie = re.sub('AAS_mre_(.*)',r'\1',dossie)
        dossie = re.sub('\/',r'-',dossie)
        dossie_url_list.append(dossie)
        url_list.append(row[2])
dossie_url_list[:5], url_list[:5]

(['d_1974.03.26',
  'd_1974.04.23',
  'vp_1975.09.03',
  'vp_1976.03.30',
  'vp_1977.10.31'],
 ['http://www.fgv.br/cpdoc/acervo/arquivo-pessoal/AAS/textual/documentos-referentes-aos-despachos-de-azeredo-da-silveira-como-ministro-das-relacoes-exteriores-com-o-presidente-da-republica-ernesto-geisel-o-do',
  'http://www.fgv.br/cpdoc/acervo/arquivo-pessoal/AAS/textual/documentos-do-tipo-lembretes-utilizados-nos-despachos-do-ministro-das-relacoes-exteriores-azeredo-da-silveira-com-o-presidente-ernesto-geisel-abor',
  'http://www.fgv.br/cpdoc/acervo/arquivo-pessoal/AAS/textual/documentos-referentes-a-atuacao-de-azeredo-da-silveira-como-ministro-das-relacoes-exteriores-nas-viagens-do-presidente-ernesto-geisel-a-franca-em-',
  'http://www.fgv.br/cpdoc/acervo/arquivo-pessoal/AAS/textual/documentos-referentes-a-atuacao-de-azeredo-da-silveira-como-ministro-das-relacoes-exteriores-na-viagem-do-presidente-ernesto-geisel-ao-japao-em-se',
  'http://www.fgv.br/cpdoc/acervo/arquivo-pessoal/AAS/textual/

# builds sql database
Classifies corpus by: id, readability and main_language.

Stores metadata in sql. Other meta-data will be placed on dossie table: dossie_id, dossie subject, date (inaccurate)

Documents with extremely low readability (less than 30% of readable phrases) are taken out of the database and listed on a file for later analysis, so that we can think of solutions in the future. Those documents are mainly manuscrits, pictures and texts with stains, scratches or drafts.

Should change in the future the lang_class column. Maybe to 2 columns: main_language, second_language.

For texts with a length of 10 **phrases**, I placed the number -1 on readability because there would be too few observations. Should make a new column with info about **word** length. Instead of the -1, I would have the estimated length and the length info.


In [29]:
doc_class = []
lang_class = 'none'
not_readable = []
percentil = int(len(fullpath_list)/100)

sql_db = os.path.join(inputs, 'cpdoc_as.sqlite')
conn = sqlite3.connect(sql_db)
cur = conn.cursor()
    
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
inserts data into sql database
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

user_input = input("Data will be erased and replaced. Continue? Type 'yes' or 'no' on your keyboard: ")
if user_input.lower() == 'yes': 
    cur.execute("DROP TABLE IF EXISTS docs")
    cur.execute('''CREATE TABLE IF NOT EXISTS docs 
               (id VARCHAR(31) PRIMARY KEY, main_language VARCHAR(10), readability DECIMAL(3,2), url LONGTEXT, body LONGTEXT
               DEFAULT NULL)''')

    ''' iterates through texts '''
    for count_doc,txt in enumerate(fullpath_list):

        ''' measures completion percentage '''
        if count_doc % percentil == 0: 
            clear_output()
            print(int(count_doc/percentil),'% done')
    #         if count_doc != 0: break

        ''' captures info about date, year, month and ids '''
        txt_date = re.sub('.*(19\d\d\.\d\d\.\d\d).*', r'\1', txt)
        txt_year = re.sub('.*(19\d\d).*', r'\1', txt)
        txt_month = re.sub('.*19\d\d\.(\d\d).*', r'\1', txt)
        txt_id = re.sub('.*AAS_mre_(.*).txt', r'\1', txt)

        dossie = re.sub('(.*)_doc_.*', r'\1', txt_id)
        url_index = dossie_url_list.index(dossie)
        url = url_list[url_index]

        ''' makes analysis in each document '''
        with open(txt, 'r', encoding=encoding_type) as f:
            txt_body = f.read()

            ''' identifies main language and readability of each document '''
            text_split = re.split('\.|\?|\:|\,', txt_body)
            pt_count = en_count = es_count = fr_count = de_count = lang_count = total_count = 0
            for phrase in text_split:
                try: 
                    if len(re.findall("[^\W\d]", phrase)) <= 10: continue
                    language = detect(phrase)
                    total_count += 1
                except: 
                    continue
                if language == 'pt':
                    pt_count += 1
                if language == 'en':
                    en_count += 1
                if language == 'es':
                    es_count += 1
                if language == 'fr':
                    fr_count += 1
                if language == 'de':
                    de_count += 1
            lang_count = pt_count + en_count + es_count + fr_count + de_count        
            if total_count == 0: readability_ratio = 0
            else: readability_ratio = float(lang_count/total_count)
            if readability_ratio < 0.3: 
                not_readable.append(txt)
                continue
            elif total_count > 10: readability = readability_ratio
            else: readability = -1
            ''' note: with the criteria, documents might have readability but no lang_class '''
            if de_count/total_count > 0.3 and de_count >= 3: 
                lang_class = 'de'
            if fr_count/total_count > 0.3 and fr_count >= 3: 
                lang_class = 'fr'
            if es_count/total_count > 0.3 and es_count >= 3: 
                lang_class = 'es'
            if en_count/total_count > 0.3 and en_count >= 3: 
                lang_class = 'en'
            if pt_count/total_count > 0.3 and pt_count >= 3: 
                lang_class = 'pt'

            ''' inserts data into sql '''
            query = "INSERT INTO docs VALUES (?,?,?,?,?)"
            cur.execute(query, (txt_id, lang_class, readability, url, txt_body))
else:
    print('Table wasnot created/replaced')
        
conn.commit()
conn.close()

100 % done


### Saves list of files with too low readability

In [10]:
not_readable_files = os.path.join(outputs, "not_readable_files.txt")

with open(not_readable_files, 'w+', encoding='utf-8') as f: 
    text = f.write(not_readable[0])
    text = f.write('\r\n')
for file in not_readable[1:]:
    with open(not_readable_files, 'a+', encoding='utf-8') as f:
        text = f.write(file)
        text = f.write('\r\n')